### Data Download

In [1]:
# Creating Kaggle.json file with username and token
import os
import json

filename = 'kaggle.json'

with open(filename, 'w') as file:
    json.dump({"username":"neilarmstrong","key":"8178685d166277cf1cab180773e875d2"}, file)

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

100%|█████████████████████████████████████████| 199k/199k [00:00<00:00, 522kB/s]
100%|█████████████████████████████████████████| 199k/199k [00:00<00:00, 492kB/s]


In [4]:
!unzip house-prices-advanced-regression-techniques.zip -d house_prices_advanced_regression_techniques

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: house_prices_advanced_regression_techniques/data_description.txt  
  inflating: house_prices_advanced_regression_techniques/sample_submission.csv  
  inflating: house_prices_advanced_regression_techniques/test.csv  
  inflating: house_prices_advanced_regression_techniques/train.csv  


### Data Wranggling

#### Import libraries and packages

In [5]:
import joblib
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# for model
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# for feature-engineering
from sklearn.preprocessing import StandardScaler
from feature_engine import missing_data_imputers as mdi
from feature_engine.discretisers import EqualFrequencyDiscretiser
from feature_engine import categorical_encoders as ce

In [6]:
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

#### Load datasets

In [7]:
data = pd.read_csv('house_prices_advanced_regression_techniques/train.csv')
train = data.copy(deep=True)
print(train.shape)
train.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### Explore

In [8]:
def explore_dataset(dataset):
    df = dataset.copy(deep=True)
    nullvals = df.isnull().sum()
    nonnulls = df.notnull().sum()
    datatype = df.dtypes
    properties = pd.Series()
    for i in df.columns.tolist():
        if pd.api.types.is_object_dtype(df[i]):
            properties[i] = df[i].unique().tolist()
        elif pd.api.types.is_numeric_dtype(df[i]):
            properties[i] = round(df[i].describe(),2).tolist()
        elif pd.api.types.is_datetime64_any_dtype(df[i]):
            properties[i] = [df[i].min().strftime(format='%Y-%m-%d'), df[i].max().strftime(format='%Y-%m-%d')]
        elif pd.api.types.is_categorical_dtype(df[i]):
            properties[i] = list(df[i].unique())
    explore = pd.concat([df.dtypes, df.isnull().sum(), df.notnull().sum(), properties], axis=1)
    explore.columns = ['dtypes', 'Missing_Vals', 'Total_Vals', 'Properties']
    return explore

In [9]:
explore_dataset(train)

/home/nm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,dtypes,Missing_Vals,Total_Vals,Properties
Id,int64,0,1460,"[1460.0, 730.5, 421.61, 1.0, 365.75, 730.5, 10..."
MSSubClass,int64,0,1460,"[1460.0, 56.9, 42.3, 20.0, 20.0, 50.0, 70.0, 1..."
MSZoning,object,0,1460,"[RL, RM, C (all), FV, RH]"
LotFrontage,float64,259,1201,"[1201.0, 70.05, 24.28, 21.0, 59.0, 69.0, 80.0,..."
LotArea,int64,0,1460,"[1460.0, 10516.83, 9981.26, 1300.0, 7553.5, 94..."
Street,object,0,1460,"[Pave, Grvl]"
Alley,object,1369,91,"[nan, Grvl, Pave]"
LotShape,object,0,1460,"[Reg, IR1, IR2, IR3]"
LandContour,object,0,1460,"[Lvl, Bnk, Low, HLS]"
Utilities,object,0,1460,"[AllPub, NoSeWa]"


From the description above, we determined the following attributes and actions to be performed for each column:

|Sr. #|ColName|Comments|attribute|
|----|----|----|----|
|01 | 'Id'| remove||
|02 | 'MSSubClass'| 15 class labels|physical|
|03 | 'MSZoning' | 05 class labels|physical|
|04 | 'LotFrontage'| continuous variable, log(LotFrontage) is less ssketed and kurtic|physical|
|05 | 'LotArea'| continuous variable, highly kurtic, log(LotArea) gives better kurtosis and skew|physical|
|06 |'Street'| no variation, 99.99% obs are paved, so remove||
|07 | 'Alley'| NaNs are to be replaced with 'no alley', only ~90 are positive|physical|
|08 | 'LotShape'| 04 class labels IR2 & IR3 could be combined into 03 total class labels|physical|
|09 | 'LandContour'| 04 ordinal class labels |physical|
|10 | 'Utilities'| 04 class labels in description but only two present with all but one having same class, so remove||
|11 | 'LotConfig'| 05 class labels could be clubbed into 04 total class labels|physical|
|12 | 'LandSlope'| 03 ordinal class labels|physical|
|13 | 'Neighborhood'| 25 class labels|market|
|14 | 'Condition1'| 09 class labels, could be clubbed into 07 class labels|externality|
|15 | 'Condition2'| 99% having same condition, so remove||
|16 | 'BldgType'| 05 distict class labels, but 99% are single-family|physical|
|17 | 'HouseStyle'| 08 distinct class labels|physical|
|18 | 'OverallQual'| 10 ordinal class labels|physical|
|19 | 'OverallCond'| 09 ordinal class labels|physical|
|20 | 'YearBuilt'| continuous integer values|market|
|21 | 'YearRemodAdd'| continuous integer values|market|
|22 | 'RoofStyle'| 06 class labels|physical|
|23 | 'RoofMatl'| 08 class labels|physical|
|24 | 'Exterior1st'| 15 class labels could be clubbed into rare categories|physical|
|25 | 'Exterior2nd'| 16 class labels coulbe be clubbed into rare categories|physical|
|26 | 'MasVnrType'| 05 class labels with 8 NaNs|physical|
|27 | 'MasVnrArea'| continuous value with 8 NaNs|physical|
|28 | 'ExterQual'| 05 ordinal class labels|physical|
|29 | 'ExterCond'| 05 ordinal class labels|physical|
|30 | 'Foundation'| 06 class labels could be clubbed into rare labels|physical|
|31 | 'BsmtQual'| 05 ordinal class labels NaN are no basement|physical|
|32 | 'BsmtCond'| 05 ordinal class labels NaN are no basement|physical|
|33 | 'BsmtExposure'| 05 ordinal class labels NaN are no basement|physical|
|34 | 'BsmtFinType1'| 07 ordinal class labels NaN are no basement|physical|
|35 | 'BsmtFinSF1'| remove||
|36 | 'BsmtFinType2'| 07 ordinal class labels NaN are no basement|physical|
|37 | 'BsmtFinSF2'| remove||
|38 | 'BsmtUnfSF'| remove||
|39 | 'TotalBsmtSF'| continuous variable|physical|
|40 | 'Heating'| 05 ordinal class variable|physical|
|41 | 'HeatingQC'| 05 ordinal class variable|physical|
|42 | 'CentralAir'| binary variable|physical|
|43 | 'Electrical'| 06 class labels can be clubbed to rare labels, contains NaN|physical|
|44 | '1stFlrSF'| continuous variable, club with '2ndFlrSF', 'TotalBsmtSF', 'GrLivArea', 'GarageArea'|physical|
|45 | '2ndFlrSF'| continuous variable, club with '1stFlrSF', 'TotalBsmtSF', 'GrLivArea', 'GarageArea'|physical|
|46 | 'LowQualFinSF'| continuous variable, low variability, remove||
|47 | 'GrLivArea'| continuous variable, club with '1stFlrSF', 'TotalBsmtSF', '2ndFlrSF', 'GarageArea'|physical|
|48 | 'BsmtFullBath'| 04 ordinal class labels, club to 'FullBath', 'HalfBath', 'BsmtHalfBath'|physical| 
|49 | 'BsmtHalfBath'| 03 ordinal class labels, club to 'FullBath', 'HalfBath', 'BsmtFullBath'|physical|
|50 | 'FullBath'| 04 ordinal class labels, club to 'BsmtFullBath', 'HalfBath', 'BsmtHalfBath'|physical| 
|51 | 'HalfBath'| 03 ordinal class labels, club to 'FullBath', 'BsmtHalfBath', 'BsmtFullBath'|physical|
|52 | 'BedroomAbvGr'| 08 ordinal class labels, can be blubbed into rare|physical| 
|53 | 'KitchenAbvGr'| 04 class, little variability, so remove||
|54 | 'KitchenQual'| 04 ordinal class labels||
|55 | 'TotRmsAbvGrd'| 12 ordinal variables, clubb 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'|physical|
|56 | 'Functional'| 07 ordinal class labels could be clubbed to fewer labels|physical|
|57 | 'Fireplaces'| 04 class class labels|physical|
|58 | 'FireplaceQu'| 06 ordinal class labels, NaN means no fire place|physical|
|59 | 'GarageType'| 07 class labels, NaN means no garage|physical|
|60 | 'GarageYrBlt'| continuous integer value|market|
|61 | 'GarageFinish'| 04 ordinal class labels, NA means no garage|physical|
|62 | 'GarageCars'| 05 class labels|physical|
|63 | 'GarageArea'| continuous variable, club with '1stFlrSF', 'TotalBsmtSF', '2ndFlrSF', 'GrLivArea'|physical|
|64 | 'GarageQual'| 06 ordinal variable, NaN means no garage|physical|
|65 | 'GarageCond'| 06 ordinal variable, NaN means no garage|physical|
|66 | 'PavedDrive'| 03 class variables|physical|
|67 | 'WoodDeckSF'| continuous variable|physical|
|68 | 'OpenPorchSF'| continuous variable|physical|
|69 | 'EnclosedPorch'| continuous variable|physical|
|70 | '3SsnPorch'| continuous variable|physical|
|71 | 'ScreenPorch'| continuous variable|physical|
|72 | 'PoolArea'| continuous variable. 99.9% 0, so remove||
|73 | 'PoolQC'| 04 ordinal labels, 99.9% Nan (no pool), so remove||
|74 | 'Fence'| 05 ordinal label, NaN means no fence|physical|
|75 | 'MiscFeature'| 05 class labels, NaN means no misc. feat|physical|
|76 | 'MiscVal'| continuous variable, value corrosponding to that misc. feat|physical|
|78 | 'MoSold'| integer value, club with 'YrSold'|market|
|79 | 'YrSold'| integer value, club with 'MoSold'|market|
|80 | 'SaleType'| 09 class labels|market|
|81 | 'SaleCondition'| 06 class labels|market|

As one can see, there are ~80 variables, which is quite a high number given the no. of observations. It becomes necessary to sanitize the dataset and check the dimensionality of the dataset to return parsimonus model. Besides, There is quite a lot of repitition of information, for e.g., Total built-up area of the property is broken into '1stFlrSF', '2ndFlrSF', and so on... and yet there is a total built-up area provided. We will try to rearrange such redundant variables in a parsimonus manner.

Next steps:
1. SUM the integer variables to be combined.
2. Replace the NaN values with 0, where appropriate
3. drop the columns to be dropped, plus the redundant columns (after combining or processing into different column).

#### Cleaning

In [10]:
# sum all the areas into TotBuiltUpArea
train['TotBuiltUpArea'] = train.TotalBsmtSF.add(train['1stFlrSF']).add(train['2ndFlrSF']).add(train['GrLivArea']).add(train['GarageArea']).add(train.TotalBsmtSF)
train.drop(labels=['1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'TotalBsmtSF'], axis=1, inplace=True)

In [11]:
# sum all the rooms into TotRoomsAbvGrd
train['TotRmsAbvGrd'] = train[['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']].sum(axis=1).add(train['TotRmsAbvGrd'])
train.drop(labels=['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'], axis=1, inplace=True)

In [12]:
# Combining all porches SF into a single column and creating porches type indicator
porches = ['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']

## summing all the porch SF into a single column
train['PorchSF'] = train.iloc[:,66:71].sum(axis=1)

## Creating porch class label
for i in range(len(porches)):
    train.loc[train[porches[i]] > 0, porches[i]] = i+1

train['PorchType'] = train[['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']].sum(axis=1)

## Deleting redundant labels
train.drop(labels=porches, axis=1, inplace=True)

In [13]:
# Replace NaNs with 0
nanval_cols = ['Alley', 'BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'MiscFeature', 'FireplaceQu']
for i in nanval_cols:
    train[i].replace(np.NaN, 'No', regex=True, inplace=True)

In [14]:
# remove columns
remove_cols = ['Id', 'Street', 'Utilities', 'Condition2', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'LowQualFinSF', 'BedroomAbvGr','KitchenAbvGr', 'KitchenQual', 'PoolArea', 'PoolQC']
train.drop(labels=remove_cols, axis=1, inplace=True)

While appraising a property, recency of a transaction is material. In other words, the transaction that took place more recently gets higher weightage. This will factor in the general inflation levels that generally drive up/ down the prices.

In [15]:
train['TransactionAge'] = dt.date.today().year - train['YrSold']
train['TransactionAge'] = 1/train['TransactionAge']

In [16]:
train['AgeAtSale'] = train['YrSold'].sub(train['YearBuilt'])

train['RemodAgeAtSale'] = train['YrSold'].sub(train['YearRemodAdd'])

# NaN values to have artibarily large numbers
train['GarageAgeAtSale'] = train['YrSold'].sub(train['GarageYrBlt'].fillna(0))

train.drop(labels=['MoSold', 'YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis=1, inplace=True)

In [17]:
# Now let's observe the shape of the dataset
train.shape

(1460, 56)

In [18]:
explore_dataset(train)

/home/nm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,dtypes,Missing_Vals,Total_Vals,Properties
MSSubClass,int64,0,1460,"[1460.0, 56.9, 42.3, 20.0, 20.0, 50.0, 70.0, 1..."
MSZoning,object,0,1460,"[RL, RM, C (all), FV, RH]"
LotFrontage,float64,259,1201,"[1201.0, 70.05, 24.28, 21.0, 59.0, 69.0, 80.0,..."
LotArea,int64,0,1460,"[1460.0, 10516.83, 9981.26, 1300.0, 7553.5, 94..."
Alley,object,0,1460,"[No, Grvl, Pave]"
LotShape,object,0,1460,"[Reg, IR1, IR2, IR3]"
LandContour,object,0,1460,"[Lvl, Bnk, Low, HLS]"
LotConfig,object,0,1460,"[Inside, FR2, Corner, CulDSac, FR3]"
LandSlope,object,0,1460,"[Gtl, Mod, Sev]"
Neighborhood,object,0,1460,"[CollgCr, Veenker, Crawfor, NoRidge, Mitchel, ..."


#### Missing Value Imputation

In [19]:
# missing value imputation
## Find out Missing Values.
missing_values = train.columns[train.isnull().any()].tolist()
print('Missing values: ', missing_values)

## First we'll fill in the sole missing value of 'Electricals' column with most frequent missing label.
train['Electrical'].fillna(value=train['Electrical'].mode()[0], inplace=True)

# Replacing 'MasVnrArea' with mean value, and 
train['MasVnrArea'].fillna(value=train.MasVnrArea.mean(), inplace=True)

train['MasVnrType'].fillna(value='missing', inplace=True)

missing_values = train.columns[train.isnull().any()].tolist()
print('Missing values: ', missing_values)

Missing values:  ['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical']
Missing values:  ['LotFrontage']


LotFrontage is an important determinant of a property value, we therefore can't afford to drop it.

Since LotFrontage has ~260 missing values, it's imprudent to simply replace NaNs with mean/ median. Further, we suspect the LotFrontage is missing not at random and that there might be underlying relationships among other variables. 

In notebook: ```Ames_House_Price_Prediction_v01.ipynb```, we trained a SVR model on select variables to impute LotFrontage NaN values. Here we will simply load the pickled model and processed dataset to replace LotFrontage NaNs

Refer [LotFontage imputation](https://www.kaggle.com/ogakulov/lotfrontage-fill-in-missing-values-house-prices) for the detailed procedure.

In [20]:
# Load Preprocessed dataset with subset of variables from the dataset 
imputed_values = joblib.load('imputed_values.pkl')

# Replacing missing values of LotFrontage.
train.loc[train['LotFrontage'].isnull(),'LotFrontage'] = imputed_values

In [21]:
# Log transformation variables
log_cols = ['LotFrontage', 'LotArea']
for i in log_cols:
    train[i] = np.log(train[i])

In [22]:
if len(train.columns[train.isnull().any()].tolist())>0:
    print('Missing Values: ', train.columns[train.isnull().any()].tolist())
else:
    print('No NaN values found!')
print('='*15)
train.sample(5)

No NaN values found!


,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,SalePrice,TotBuiltUpArea,PorchSF,PorchType,TransactionAge,AgeAtSale,RemodAgeAtSale,GarageAgeAtSale
1017,120,RL,3.216560,8.668024,No,IR1,Lvl,CulDSac,Gtl,StoneBr,Norm,TwnhsE,1Story,8,5,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,CBlock,Gd,TA,Av,GLQ,Unf,GasA,Gd,Y,SBrkr,6,Typ,1,Ex,Attchd,RFn,2,TA,TA,Y,No,No,0,COD,Abnorml,187500,5725,2017,1,0.090909,25,25,25.0
12,20,RL,4.233614,9.470240,No,IR2,Lvl,Inside,Gtl,Sawyer,Norm,1Fam,1Story,5,6,Hip,CompShg,HdBoard,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,6,Typ,0,No,Detchd,Unf,1,TA,TA,Y,No,No,0,WD,Normal,144000,4000,2017,6,0.083333,46,46,46.0
721,120,RM,3.444875,8.395252,No,Reg,Lvl,Inside,Gtl,CollgCr,Norm,TwnhsE,1Story,6,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,169.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,5,Typ,0,No,Attchd,RFn,2,TA,TA,Y,No,No,0,WD,Normal,143000,3812,2015,1,0.100000,6,6,6.0
161,60,RL,4.700480,9.524275,No,IR1,Lvl,Inside,Gtl,NridgHt,Norm,1Fam,2Story,9,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,664.0,Gd,TA,PConc,Ex,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,14,Typ,2,Gd,BuiltIn,Fin,3,TA,TA,Y,No,No,0,WD,Normal,412500,9206,2011,1,0.083333,5,4,5.0
1105,60,RL,4.584967,9.413771,No,IR1,Lvl,Corner,Gtl,NoRidge,Norm,1Fam,2Story,8,5,Gable,CompShg,HdBoard,HdBoard,BrkFace,362.0,Gd,TA,PConc,Ex,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,13,Typ,2,TA,Attchd,RFn,2,TA,TA,Y,No,No,0,WD,Normal,325000,8882,2014,3,0.100000,16,15,16.0


Next, we will identify the true data type of each variable, transform that variable accordingly and create groupings for easy reference in pipelines. The purpose of this exercise is to identify the treatment of each dtype. For instance, object data type that are nominal will first need to be converted to numerical before applying any transformation.

Steps involved:

1. Identify true data type of each variable.
2. Classify the variables into, continuous values, categorical-numericalType, categorical-objectType, ordinal-numericalType, ordinal-objectType

|continuous|nominal-numericalType|nominal-objectType|ordinal-numericalType|ordinal-objectType|
|---|---|---|---|---|
|LotFrontage|MSSubClass|MSZoning|OverallQual|ExterQual|
|LotArea|PorchType|Alley|OverallCond|ExterCond|
|MasVnrArea|TransactionAge|LandContour||BsmtQual|
|Fireplaces||LotConfig||BsmtExposure|
|GarageCars||LandSlope||BsmtFinType1|
|MiscVal||Neighborhood||BsmtFinType2|
|TotBuiltUpArea||Condition1||HeatingQC|
|PorchSF||BldgType||FireplaceQu|
|AgeAtSale||HouseStyle||GarageFinish|
|||RoofStyle||GarageQual|
|||Exterior1st||GarageCond|
|||Exterior2nd||Fence|
|||MasVnrType||BsmtCond|
|||Foundation|||
|||Heating|||
|||CentralAir|||
|||Electrical|||
|||Functional|||
|||GarageType|||
|||PavedDrive|||
|||MiscFeature|||
|||SaleType|||
|||SaleCondition|||

Post that, we will develop a preprocessing pipeline that will perform following steps:
1. Rare Label Encoding of categorical-objecttype variables.
2. Transform Categorical-objectType and ordinal-objectTypes into categorical-numericalType and ordinal-numericalTypes, respectively. We will then have three variables types; actual numerical, categorical and ordinal.
3. Mean-Encode categorical variables and Ordinal-Encode ordinal variables
4. Standardize entire dataset.

We plan to train three models on the data _viz_., Lasso, ANN and LightGBT. For LightGBT we require additional step of feature selection.

the models will be evaluated, predicted and interpreted

Lastly, we will stack up all the models using RidgeCV as meta model.

#### Data Pre-Processing

In [23]:
X = train.drop(labels=['SalePrice'], axis=1)
y = train['SalePrice']

col_names = train.columns.tolist()
col_names.remove('SalePrice')

In [24]:
continuous = ['LotFrontage', 'LotArea', 'MasVnrArea', 'Fireplaces', 'GarageCars', 'MiscVal', 'TotBuiltUpArea', 'PorchSF', 'AgeAtSale']
nom_int = ['MSSubClass', 'PorchType', 'TransactionAge']
nom_obj = ['MSZoning', 'Alley', 'LandContour', 'LotShape','LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition']
ord_int = ['OverallQual', 'OverallCond']
ord_obj = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence']

### Trainig Linear Regularised Model (LASSO)

Data processing steps in the pipeline:

1. RareLabelsEncoding of nominal labels
2. MeanEncoding of nominal-Objecttype variables
3. OrdinalEncoding of ordinal-ObjectType variables
4. Standardizing the dataset

In [25]:
pipe = Pipeline([
    ('rareLabels', ce.RareLabelCategoricalEncoder(n_categories=5, variables=nom_obj)),
    ('mean_encode', ce.MeanCategoricalEncoder(variables=nom_obj)),
    ('ordinal_encode', ce.OrdinalCategoricalEncoder(variables=ord_obj)), 
    ('std_scaler', StandardScaler())
])

pipe.fit(X, y)

/home/nm/miniconda3/lib/python3.7/site-packages/feature_engine/categorical_encoders.py:786: UserWarning: The number of unique categories for variable MSZoning is less than that indicated in n_categories. Thus, all categories will be considered frequent
  "n_categories. Thus, all categories will be considered frequent".format(var))
/home/nm/miniconda3/lib/python3.7/site-packages/feature_engine/categorical_encoders.py:786: UserWarning: The number of unique categories for variable Alley is less than that indicated in n_categories. Thus, all categories will be considered frequent
  "n_categories. Thus, all categories will be considered frequent".format(var))
/home/nm/miniconda3/lib/python3.7/site-packages/feature_engine/categorical_encoders.py:786: UserWarning: The number of unique categories for variable LandContour is less than that indicated in n_categories. Thus, all categories will be considered frequent
  "n_categories. Thus, all categories will be considered frequent".format(var))
/

Pipeline(steps=[('rareLabels',
                 RareLabelCategoricalEncoder(n_categories=5,
                                             variables=['MSZoning', 'Alley',
                                                        'LandContour',
                                                        'LotShape', 'LotConfig',
                                                        'LandSlope',
                                                        'Neighborhood',
                                                        'Condition1',
                                                        'BldgType',
                                                        'HouseStyle',
                                                        'RoofStyle', 'RoofMatl',
                                                        'Exterior1st',
                                                        'Exterior2nd',
                                                        'MasVnrType',
                                     

In [26]:
X = pd.DataFrame(pipe.transform(X), columns=col_names)

In [27]:
model_lasso = Lasso(alpha=0.1, random_state=0)

mse = 0
mae = 0
rsq = []

k_fold = KFold(n_splits=7, shuffle=True, random_state=0)

for trn, tst in k_fold.split(X):
    model_lasso.fit(X.iloc[trn,:], y.iloc[trn])
    y_pred = model_lasso.predict(X.iloc[tst,:])
    mse += mean_squared_error(y.iloc[tst], y_pred)
    mae += mean_absolute_error(y.iloc[tst], y_pred)
    rsq.append(r2_score(y.iloc[tst], y_pred))

print('Root Mean Squarred Error: %.4f' % ((mse ** 0.5)/7))
print('Mean Absolute Error:      %.4f' % (mae/7))
print('R2_score:                 %.4f' % np.mean(rsq))

/home/nm/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 597133138982.4988, tolerance: 796553503.1657763
  positive)
/home/nm/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 535726540514.5486, tolerance: 778497763.9959022
  positive)
/home/nm/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 556929278934.3384, tolerance: 777525502.1939697
  positive)
/home/nm/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Root Mean Squarred Error: 13260.1877
Mean Absolute Error:      21037.1142
R2_score:                 0.8065


/home/nm/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 661175070027.3547, tolerance: 824916992.2819811
  positive)
